In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import tools as ts
from encode import encode_train_test
from compress import compress_24m
from generate_train_from_24m import generate_train_salesVolume, generate_train_popularity

In [3]:
#首先对训练集和测试集都进行编码
path = '../Data/'
train_data, eval_data = encode_train_test(path+'Handle/sales+search.csv', path+'evaluation_public.csv')

#将每个省，每种车型的数据压缩成一行数据
compress_data = compress_24m(train_data)

#按照时间滑窗12个月生成训练集
train_data = generate_train_salesVolume(compress_data)
train_data_2 = generate_train_popularity(compress_data)

#输出训练集
'''train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)'''

range(0, 1320)


"train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)"

In [40]:
base_columns = ['province', 'adcode', 'model', 'bodyType']
fix1 = 'salesVolume'
fix2 = 'popularity'
final_columns = []
for i in range(1, 13):
    final_columns.append(fix1+str(i))
    final_columns.append(fix2+str(i))
    
#将每个值都变为int型
train_data = ts.transform_type(train_data, train_data.columns, 'float')
train_data_2 = ts.transform_type(train_data_2, train_data_2.columns, 'float')

#训练模型
model1= xgb.XGBRegressor(n_estimators=3000)
train_x = train_data.drop(['pred'], axis=1)
train_y = train_data['pred']
model1.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='rmse', verbose=True)
model1.save_model('model_1_salesVolume')

model2 = xgb.XGBRegressor(n_estimators=3000)
train_x = train_data_2.drop(['pred'], axis=1)
train_y = train_data_2['pred']
model2.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='rmse', verbose=True)
model2.save_model('model_1_popularity')
    
#计算出2018年1月的数据
change_columns = []
for i in range(13, 25):
    change_columns.append(fix1+str(i))
    change_columns.append(fix2+str(i))
test_data_1 = compress_data[base_columns+change_columns]
test_data_1.columns = base_columns+final_columns
test_data_1 = ts.transform_type(test_data_1, test_data_1.columns, 'float')
test_data_1 = test_data_1.reset_index(drop=True)
print(test_data_1.shape)

model1 = xgb.XGBRegressor()
model1.load_model('model_1_salesVolume')
model2 = xgb.XGBRegressor()
model2.load_model('model_1_popularity')
pred_1_1 = pd.DataFrame(model1.predict(test_data_1), columns=['pred1'])
pred_1_2 = pd.DataFrame(model2.predict(test_data_1), columns=['pred2'])
print(test_data_1.shape)
result_1 = pd.concat([test_data_1, pred_1_1, pred_1_2], axis=1)

#计算出2018年2月的数据
test_data_2 = result_1.drop(['salesVolume1', 'salesVolume2'], axis=1)
test_data_2.columns = base_columns+final_columns
pred_2_1 = pd.DataFrame(model1.predict(test_data_2), columns=['pred1'])
pred_2_2 = pd.DataFrame(model2.predict(test_data_2), columns=['pred2'])
result_2 = pd.concat([test_data_2, pred_2_1, pred_2_2], axis=1)

#计算出2018年3月的数据
test_data_3 = result_2.drop(['salesVolume1', 'salesVolume2'], axis=1)
test_data_3.columns = base_columns+final_columns
pred_3_1 = pd.DataFrame(model1.predict(test_data_3), columns=['pred1'])
pred_3_2 = pd.DataFrame(model2.predict(test_data_3), columns=['pred2'])
result_3 = pd.concat([test_data_3, pred_3_1, pred_3_2], axis=1)

#计算出2018年4月的数据
test_data_4 = result_3.drop(['salesVolume1', 'salesVolume2'], axis=1)
test_data_4.columns = base_columns+final_columns
pred_4_1 = pd.DataFrame(model1.predict(test_data_4), columns=['pred1'])
pred_4_2 = pd.DataFrame(model2.predict(test_data_4), columns=['pred2'])
result_4 = pd.concat([test_data_4, pred_4_1, pred_4_2], axis=1)

eval_columns = base_columns + ['pred1']
predict_Jan = result_1[eval_columns]
predict_Jan['regMonth'] = 1
predict_Feb = result_2[eval_columns]
predict_Feb['regMonth'] = 2
predict_Mar = result_3[eval_columns]
predict_Mar['regMonth'] = 3
predict_Apr = result_4[eval_columns]
predict_Apr['regMonth'] = 4

print(predict_Jan)

[0]	validation_0-rmse:873.16858
[1]	validation_0-rmse:796.79681
[2]	validation_0-rmse:728.35107
[3]	validation_0-rmse:667.36920
[4]	validation_0-rmse:612.76935
[5]	validation_0-rmse:563.93555
[6]	validation_0-rmse:520.67798
[7]	validation_0-rmse:482.16470
[8]	validation_0-rmse:447.17990
[9]	validation_0-rmse:416.87256
[10]	validation_0-rmse:389.88882
[11]	validation_0-rmse:366.50855
[12]	validation_0-rmse:345.51111
[13]	validation_0-rmse:327.12787
[14]	validation_0-rmse:311.07996
[15]	validation_0-rmse:296.88837
[16]	validation_0-rmse:284.92050
[17]	validation_0-rmse:274.15616
[18]	validation_0-rmse:264.80359
[19]	validation_0-rmse:257.18448
[20]	validation_0-rmse:250.57240
[21]	validation_0-rmse:244.68742
[22]	validation_0-rmse:239.90976
[23]	validation_0-rmse:235.67322
[24]	validation_0-rmse:232.16760
[25]	validation_0-rmse:229.04593
[26]	validation_0-rmse:225.61130
[27]	validation_0-rmse:223.39096
[28]	validation_0-rmse:221.43335
[29]	validation_0-rmse:219.11479
[30]	validation_0-rm

[245]	validation_0-rmse:152.37703
[246]	validation_0-rmse:152.07257
[247]	validation_0-rmse:151.80627
[248]	validation_0-rmse:151.57248
[249]	validation_0-rmse:151.43851
[250]	validation_0-rmse:151.28400
[251]	validation_0-rmse:151.03207
[252]	validation_0-rmse:150.83543
[253]	validation_0-rmse:150.72212
[254]	validation_0-rmse:150.51025
[255]	validation_0-rmse:150.38055
[256]	validation_0-rmse:150.00995
[257]	validation_0-rmse:149.80295
[258]	validation_0-rmse:149.60309
[259]	validation_0-rmse:149.57156
[260]	validation_0-rmse:149.52266
[261]	validation_0-rmse:149.48940
[262]	validation_0-rmse:149.40965
[263]	validation_0-rmse:149.25501
[264]	validation_0-rmse:149.10451
[265]	validation_0-rmse:148.98114
[266]	validation_0-rmse:148.77327
[267]	validation_0-rmse:148.67145
[268]	validation_0-rmse:148.51279
[269]	validation_0-rmse:148.36063
[270]	validation_0-rmse:148.21298
[271]	validation_0-rmse:148.16112
[272]	validation_0-rmse:147.95859
[273]	validation_0-rmse:147.77275
[274]	validati

[486]	validation_0-rmse:125.76099
[487]	validation_0-rmse:125.54079
[488]	validation_0-rmse:125.47241
[489]	validation_0-rmse:125.37764
[490]	validation_0-rmse:125.29838
[491]	validation_0-rmse:125.23741
[492]	validation_0-rmse:125.07269
[493]	validation_0-rmse:124.99836
[494]	validation_0-rmse:124.85262
[495]	validation_0-rmse:124.75301
[496]	validation_0-rmse:124.60458
[497]	validation_0-rmse:124.51907
[498]	validation_0-rmse:124.43211
[499]	validation_0-rmse:124.39214
[500]	validation_0-rmse:124.35463
[501]	validation_0-rmse:124.24754
[502]	validation_0-rmse:124.14796
[503]	validation_0-rmse:124.07201
[504]	validation_0-rmse:123.99828
[505]	validation_0-rmse:123.97888
[506]	validation_0-rmse:123.92480
[507]	validation_0-rmse:123.83498
[508]	validation_0-rmse:123.76271
[509]	validation_0-rmse:123.67878
[510]	validation_0-rmse:123.60986
[511]	validation_0-rmse:123.55927
[512]	validation_0-rmse:123.50269
[513]	validation_0-rmse:123.42168
[514]	validation_0-rmse:123.33698
[515]	validati

[727]	validation_0-rmse:110.20498
[728]	validation_0-rmse:110.14890
[729]	validation_0-rmse:110.06762
[730]	validation_0-rmse:110.01208
[731]	validation_0-rmse:109.93922
[732]	validation_0-rmse:109.90035
[733]	validation_0-rmse:109.82848
[734]	validation_0-rmse:109.77481
[735]	validation_0-rmse:109.73325
[736]	validation_0-rmse:109.66167
[737]	validation_0-rmse:109.61053
[738]	validation_0-rmse:109.47876
[739]	validation_0-rmse:109.42762
[740]	validation_0-rmse:109.37265
[741]	validation_0-rmse:109.33903
[742]	validation_0-rmse:109.28863
[743]	validation_0-rmse:109.28140
[744]	validation_0-rmse:109.22846
[745]	validation_0-rmse:109.17429
[746]	validation_0-rmse:109.12150
[747]	validation_0-rmse:109.09654
[748]	validation_0-rmse:109.05065
[749]	validation_0-rmse:108.94637
[750]	validation_0-rmse:108.90281
[751]	validation_0-rmse:108.86432
[752]	validation_0-rmse:108.80722
[753]	validation_0-rmse:108.78143
[754]	validation_0-rmse:108.75672
[755]	validation_0-rmse:108.71062
[756]	validati

[969]	validation_0-rmse:99.12246
[970]	validation_0-rmse:99.08734
[971]	validation_0-rmse:99.03204
[972]	validation_0-rmse:98.98621
[973]	validation_0-rmse:98.94434
[974]	validation_0-rmse:98.89997
[975]	validation_0-rmse:98.80778
[976]	validation_0-rmse:98.77966
[977]	validation_0-rmse:98.74492
[978]	validation_0-rmse:98.68668
[979]	validation_0-rmse:98.65731
[980]	validation_0-rmse:98.62491
[981]	validation_0-rmse:98.60610
[982]	validation_0-rmse:98.59335
[983]	validation_0-rmse:98.54636
[984]	validation_0-rmse:98.48856
[985]	validation_0-rmse:98.45243
[986]	validation_0-rmse:98.43623
[987]	validation_0-rmse:98.42411
[988]	validation_0-rmse:98.37671
[989]	validation_0-rmse:98.37394
[990]	validation_0-rmse:98.35310
[991]	validation_0-rmse:98.30327
[992]	validation_0-rmse:98.27456
[993]	validation_0-rmse:98.23978
[994]	validation_0-rmse:98.19929
[995]	validation_0-rmse:98.15010
[996]	validation_0-rmse:98.12246
[997]	validation_0-rmse:98.08469
[998]	validation_0-rmse:98.04368
[999]	vali

...

[2041]	validation_0-rmse:262.80899
[2042]	validation_0-rmse:262.72003
[2043]	validation_0-rmse:262.63882
[2044]	validation_0-rmse:262.55701
[2045]	validation_0-rmse:262.40747
[2046]	validation_0-rmse:262.40006
[2047]	validation_0-rmse:262.34875
[2048]	validation_0-rmse:262.25803
[2049]	validation_0-rmse:262.17679
[2050]	validation_0-rmse:262.08890
[2051]	validation_0-rmse:262.00537
[2052]	validation_0-rmse:261.88504
[2053]	validation_0-rmse:261.84793
[2054]	validation_0-rmse:261.80228
[2055]	validation_0-rmse:261.79861
[2056]	validation_0-rmse:261.75058
[2057]	validation_0-rmse:261.42056
[2058]	validation_0-rmse:261.35666
[2059]	validation_0-rmse:261.25296
[2060]	validation_0-rmse:261.19000
[2061]	validation_0-rmse:261.08316
[2062]	validation_0-rmse:260.94656
[2063]	validation_0-rmse:260.88004
[2064]	validation_0-rmse:260.78564
[2065]	validation_0-rmse:260.70956
[2066]	validation_0-rmse:260.62772
[2067]	validation_0-rmse:260.54562
[2068]	validation_0-rmse:260.45172
[2069]	validation_0-

[2276]	validation_0-rmse:245.85835
[2277]	validation_0-rmse:245.81863
[2278]	validation_0-rmse:245.75279
[2279]	validation_0-rmse:245.62012
[2280]	validation_0-rmse:245.52125
[2281]	validation_0-rmse:245.51741
[2282]	validation_0-rmse:245.45190
[2283]	validation_0-rmse:245.35635
[2284]	validation_0-rmse:245.27820
[2285]	validation_0-rmse:245.15697
[2286]	validation_0-rmse:245.03232
[2287]	validation_0-rmse:244.94369
[2288]	validation_0-rmse:244.87695
[2289]	validation_0-rmse:244.77739
[2290]	validation_0-rmse:244.67424
[2291]	validation_0-rmse:244.62122
[2292]	validation_0-rmse:244.54099
[2293]	validation_0-rmse:244.48976
[2294]	validation_0-rmse:244.46674
[2295]	validation_0-rmse:244.39535
[2296]	validation_0-rmse:244.28812
[2297]	validation_0-rmse:244.22960
[2298]	validation_0-rmse:244.22308
[2299]	validation_0-rmse:244.20528
[2300]	validation_0-rmse:244.13103
[2301]	validation_0-rmse:243.89291
[2302]	validation_0-rmse:243.83011
[2303]	validation_0-rmse:243.77292
[2304]	validation_0-

[2511]	validation_0-rmse:229.70999
[2512]	validation_0-rmse:229.65678
[2513]	validation_0-rmse:229.60053
[2514]	validation_0-rmse:229.53700
[2515]	validation_0-rmse:229.43556
[2516]	validation_0-rmse:229.33157
[2517]	validation_0-rmse:229.27028
[2518]	validation_0-rmse:229.26616
[2519]	validation_0-rmse:229.24713
[2520]	validation_0-rmse:229.20235
[2521]	validation_0-rmse:229.14975
[2522]	validation_0-rmse:229.06082
[2523]	validation_0-rmse:229.01979
[2524]	validation_0-rmse:228.97545
[2525]	validation_0-rmse:228.86372
[2526]	validation_0-rmse:228.80527
[2527]	validation_0-rmse:228.78258
[2528]	validation_0-rmse:228.77979
[2529]	validation_0-rmse:228.71252
[2530]	validation_0-rmse:228.66946
[2531]	validation_0-rmse:228.61148
[2532]	validation_0-rmse:228.53583
[2533]	validation_0-rmse:228.49634
[2534]	validation_0-rmse:228.44554
[2535]	validation_0-rmse:228.36572
[2536]	validation_0-rmse:228.31592
[2537]	validation_0-rmse:228.17377
[2538]	validation_0-rmse:228.12991
[2539]	validation_0-

[2746]	validation_0-rmse:216.45905
[2747]	validation_0-rmse:216.44296
[2748]	validation_0-rmse:216.42061
[2749]	validation_0-rmse:216.36760
[2750]	validation_0-rmse:216.31947
[2751]	validation_0-rmse:216.24033
[2752]	validation_0-rmse:216.20084
[2753]	validation_0-rmse:216.11368
[2754]	validation_0-rmse:216.06943
[2755]	validation_0-rmse:216.01315
[2756]	validation_0-rmse:215.98644
[2757]	validation_0-rmse:215.92389
[2758]	validation_0-rmse:215.80667
[2759]	validation_0-rmse:215.76190
[2760]	validation_0-rmse:215.62619
[2761]	validation_0-rmse:215.54338
[2762]	validation_0-rmse:215.48015
[2763]	validation_0-rmse:215.47649
[2764]	validation_0-rmse:215.42960
[2765]	validation_0-rmse:215.39604
[2766]	validation_0-rmse:215.36482
[2767]	validation_0-rmse:215.32533
[2768]	validation_0-rmse:215.31886
[2769]	validation_0-rmse:215.26866
[2770]	validation_0-rmse:215.24545
[2771]	validation_0-rmse:215.21640
[2772]	validation_0-rmse:215.15512
[2773]	validation_0-rmse:215.08824
[2774]	validation_0-

[2981]	validation_0-rmse:204.36618
[2982]	validation_0-rmse:204.30064
[2983]	validation_0-rmse:204.21483
[2984]	validation_0-rmse:204.11690
[2985]	validation_0-rmse:204.05188
[2986]	validation_0-rmse:203.97832
[2987]	validation_0-rmse:203.94031
[2988]	validation_0-rmse:203.85702
[2989]	validation_0-rmse:203.83551
[2990]	validation_0-rmse:203.75963
[2991]	validation_0-rmse:203.68414
[2992]	validation_0-rmse:203.65652
[2993]	validation_0-rmse:203.63796
[2994]	validation_0-rmse:203.62430
[2995]	validation_0-rmse:203.59796
[2996]	validation_0-rmse:203.58043
[2997]	validation_0-rmse:203.56935
[2998]	validation_0-rmse:203.54576
[2999]	validation_0-rmse:203.52464
(1320, 28)
(1320, 28)
      province  adcode  model  bodyType        pred1  regMonth
0          3.0     0.0    0.0       3.0   595.413391         1
1         12.0     1.0    0.0       3.0   330.464172         1
2          7.0     2.0    0.0       3.0    98.598228         1
3          2.0     3.0    0.0       3.0    55.791183         

In [44]:
#将预测的值和上交集拼接在一起
predict = pd.concat([predict_Jan, predict_Feb, predict_Mar, predict_Apr], axis=0)
output = eval_data.merge(predict, on=['province', 'adcode', 'model', 'regMonth'])
output = output[['id', 'pred1']]
output.columns = ['id', 'forecastVolum']
output['forecastVolum'] = output['forecastVolum'].astype('int')
output['forecastVolum'] = output['forecastVolum'].apply(lambda x: abs(x))
output.to_csv('../Data/Final/result_1.csv', index=False)